In [11]:
import torch

In [22]:
data_processed = 'data_processed'

In [23]:
activities = torch.load(data_processed + '/activities.pt')
poses = torch.load(data_processed + '/poses.pt')

/tmp/ipykernel_2380161/4058910880.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  activities = torch.load(data_processed + '/activities.pt')
/tmp/ipykernel_2380161/40589

In [44]:
class_count =activities.sum(dim=0)
# output the most common activities by sorting the class count
print(class_count)
_, indices = torch.sort(class_count, descending=True)
print(indices)


tensor([4.9000e+01, 2.4967e+04, 1.3155e+04, 4.7700e+02, 5.4240e+03, 3.0000e+01,
        2.8000e+01, 1.0950e+03, 3.2300e+02, 9.8000e+01, 6.8000e+01, 8.3000e+01,
        4.0000e+00, 4.9000e+02, 7.3000e+02, 2.0630e+03, 1.6250e+03, 4.9300e+02,
        9.0000e+00, 1.0000e+00, 1.4800e+02, 8.9000e+01, 2.0000e+02, 3.3000e+01,
        1.3900e+02, 9.5000e+01, 2.4710e+03, 2.6000e+01, 8.0000e+00, 5.7840e+03,
        1.3690e+03, 2.0000e+01, 3.1810e+03, 2.2000e+01, 2.9700e+02, 1.1400e+02,
        6.6000e+01, 9.5000e+01, 0.0000e+00, 8.7000e+01, 2.0320e+03, 1.8000e+02,
        5.1000e+01, 9.7000e+01, 8.3600e+02, 2.0000e+00, 1.5000e+01, 4.3000e+01,
        1.1200e+02, 4.8000e+01, 8.0000e+00, 3.1000e+01, 1.1000e+01, 4.0000e+00,
        1.8500e+02, 5.4000e+01, 5.4560e+03, 1.0000e+01, 4.9184e+04, 3.2700e+02,
        2.2000e+01, 6.2463e+04, 5.1800e+02, 7.0000e+00, 1.5600e+02, 4.7450e+03,
        1.6000e+01, 9.7000e+01, 7.1000e+01, 5.1500e+02, 1.6300e+02, 4.2300e+02,
        1.2800e+02, 1.7900e+02, 1.3160e+

In [48]:
# find if there are any activities that do not have labels 61 and 58
print(activities[:, 61].sum())
print(activities[:, 58].sum())

print(activities[:, 61].sum() + activities[:, 58].sum())

tensor(62463.)
tensor(49184.)
tensor(111647.)


In [64]:
# show activities that do not have labels 61 and 58
activities_no_58 = activities[activities[:, 58] == 0]
activities_no_61 = activities_no_58[activities_no_58[:, 61] == 0]
activities_no_61_count = activities_no_61.sum(dim=0)

In [68]:
import torch
from torch_geometric.nn import DataParallel  # Use PyG DataParallel
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import wandb
from hik.data.scene import Scene
from models import GINE  # Import the GINE model
from data_prep import generate_dataloaders

# Initialize WandB
# wandb.init(project="main-gine-inference", entity="kaisar-dauletbek")

# Configuration
config = {
    "batch_size": 1028,
    "hidden_dim": 512,
    "input_dim": 4,
    "num_layers": 8,
    "lr": 0.001,
    "model_save_path": "best_main_gine_model.pth"
}
# wandb.config.update(config)

# Load dataset and create dataloaders
graph_dataset, _, _, test_loader = generate_dataloaders(config["batch_size"])

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Output dimension based on dataset
output_dim = graph_dataset[0].y.size(1)

# Load the sit_stand model
sit_stand_model = GINE(
    num_features=3,
    num_classes=3,
    hidden_channels=256,
    num_layers=8,
    use_batch_norm=True
).to(device)
sit_stand_model.load_state_dict(torch.load("best_sit_stand.pth"))
sit_stand_model.eval()

# Load the main model
model = GINE(
    num_features=config["input_dim"],
    num_classes=output_dim,
    hidden_channels=128,
    num_layers=config["num_layers"],
    use_batch_norm=True
).to(device)
model.load_state_dict(torch.load(config["model_save_path"]))
model.eval()

# Inference Script for Test Split
def inference_on_test_split(model, test_loader):
    """
    Perform inference on the test split and calculate multiclass accuracy.

    Args:
        model: Trained GINE model.
        test_loader: DataLoader for the test set.

    Returns:
        multiclass_accuracy: Accuracy of the predictions across multiple classes.
    """
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Inference on Test Set"):
            # Move batch to device
            batch = batch.to(device)

            # Get sit/stand predictions from the pre-trained model
            sit_stand_out = sit_stand_model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            sit_stand_pred = torch.argmax(sit_stand_out, dim=1)
            sit_stand_pred = torch.repeat_interleave(sit_stand_pred, 145, dim=0).unsqueeze(0).T

            # Add sit/stand predictions to input features
            batch.x = torch.cat((batch.x, sit_stand_pred), 1)

            # Get predictions from the main model
            out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)

            # Convert logits to class predictions
            preds = torch.argmax(torch.sigmoid(out), dim=1)

            # Accumulate predictions and labels
            all_preds.append(preds.cpu())
            all_labels.append(torch.argmax(batch.y, dim=1).cpu())  # Convert one-hot to class indices

            # Calculate accuracy for this batch
            correct += (preds == torch.argmax(batch.y, dim=1)).sum().item()
            total += batch.y.size(0)

    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # Calculate overall accuracy
    multiclass_accuracy = correct / total
    print(f"Multiclass Accuracy on Test Set: {multiclass_accuracy:.4f}")

    # Optionally log the accuracy using WandB
    wandb.log({"test_multiclass_accuracy": multiclass_accuracy})

    return multiclass_accuracy


# Perform inference and calculate test accuracy
test_accuracy = inference_on_test_split(model, test_loader)
print(f"Final Test Multiclass Accuracy: {test_accuracy:.4f}")


Filtered dataset size: 62067


/tmp/ipykernel_2380161/4272263145.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sit_stand_model.load_state_dict(torch.load("best_sit_stand.pth"))
/tmp/ipykernel_23801

Multiclass Accuracy on Test Set: 0.6097
Final Test Multiclass Accuracy: 0.6097


In [1]:
import torch
from torch_geometric.nn import DataParallel  # Use PyG DataParallel
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import wandb
from hik.data.scene import Scene
from models import GraphTransformer  # Import the GINE model
from data_prep import generate_dataloaders

# Initialize WandB
# wandb.init(project="main-gine-inference", entity="kaisar-dauletbek")

# Configuration
config = {
    "batch_size": 128,
    "hidden_dim": 256,
    "input_dim": 3,
    "num_heads": 16,
    "lr": 0.001,
    "epochs": 100,
    "model_save_path": "best_model.pth"
}
# wandb.config.update(config)

# Load dataset and create dataloaders
graph_dataset, _, _, test_loader = generate_dataloaders(config["batch_size"])

/home/kd/Documents/graphml/Graph_Machine_Learning_ProjectX/data_prep.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_dataset = torch.load('data_processed/graphs.

Filtered dataset size: 78065


In [4]:


# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Output dimension based on dataset
output_dim = graph_dataset[0].y.size(1)

# # Load the sit_stand model
# sit_stand_model = GINE(
#     num_features=3,
#     num_classes=3,
#     hidden_channels=256,
#     num_layers=8,
#     use_batch_norm=True
# ).to(device)
# sit_stand_model.load_state_dict(torch.load("best_sit_stand.pth"))
# sit_stand_model.eval()

# Load the main model

model = GraphTransformer(
    config["input_dim"],
    config["hidden_dim"],
    output_dim,
    config["num_heads"]
).to(device)

model.load_state_dict(torch.load(config["model_save_path"]))
model.eval()

# Inference Script for Test Split
def inference_on_test_split(model, test_loader):
    """
    Perform inference on the test split and calculate multiclass accuracy.

    Args:
        model: Trained GINE model.
        test_loader: DataLoader for the test set.

    Returns:
        multiclass_accuracy: Accuracy of the predictions across multiple classes.
    """
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Inference on Test Set"):
            # Move batch to device
            batch = batch.to(device)

            # Get sit/stand predictions from the pre-trained model
            # sit_stand_out = sit_stand_model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            # sit_stand_pred = torch.argmax(sit_stand_out, dim=1)
            # sit_stand_pred = torch.repeat_interleave(sit_stand_pred, 145, dim=0).unsqueeze(0).T

            # Add sit/stand predictions to input features
            # batch.x = torch.cat((batch.x, sit_stand_pred), 1)

            # Get predictions from the main model
            # redefine batch without edge_attr
            out = model(batch)

            # Convert logits to class predictions
            # preds = torch.argmax(torch.sigmoid(out), dim=1)
            preds = torch.argmax(out, dim=1)

            # Accumulate predictions and labels
            all_preds.append(preds.cpu())
            all_labels.append(torch.argmax(batch.y, dim=1).cpu())  # Convert one-hot to class indices

            # Calculate accuracy for this batch
            correct += (preds == torch.argmax(batch.y, dim=1)).sum().item()
            total += batch.y.size(0)

    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # Calculate overall accuracy
    multiclass_accuracy = correct / total
    print(f"Multiclass Accuracy on Test Set: {multiclass_accuracy:.4f}")

    # Optionally log the accuracy using WandB
    # wandb.log({"test_multiclass_accuracy": multiclass_accuracy})

    return multiclass_accuracy


# Perform inference and calculate test accuracy
test_accuracy = inference_on_test_split(model, test_loader)
print(f"Final Test Multiclass Accuracy: {test_accuracy:.4f}")


/tmp/ipykernel_3223708/4232834263.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config["model_save_path"]))
Inference on Test Set: 10

Multiclass Accuracy on Test Set: 0.1447
Final Test Multiclass Accuracy: 0.1447


In [73]:
graphs = torch.load('data_processed/graphs.pt')

/tmp/ipykernel_2380161/2713139490.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graphs = torch.load('data_processed/graphs.pt')


In [80]:
graphs[0].edge_attr[0].float()

tensor([0.1166])